# Flightops - Data Acquisition, Loading and Pre-processing

## Extracting Data from BTS
Flight data for 2018–2019 is sourced from the BTS TranStats repository using an automated downloader script:
- `download_bts_ontime.py` fetches monthly On-Time Performance files and extracts them into the project directory.
- Run this command from the repo root to download the necessary files: *python notebooks/download_bts_ontime.py*

## Load and Clean Data

Data cleaning is one of the most critical (and usually most tedious) part of data analysis. But the effort is always worth it because, as the old saying goes, **garbage in, garbage out!**

environment: conda activate datasci311

### Set stable root directory for relative paths in VSCode

VS Code + Jupyter has notoriously unstable CWDs because it is flexible in where it runs notebooks from (e.g. the workspace root, a previously cached directory, a kernel opened *before* the repo was opened...). It's great that VSC has this flexibility, but it presents a weakness when making repos available in CI/GitHub. I've removed the ambiguity by controlling the CWD to make relative paths stable.

In [ ]:
from pathlib import Path
import pandas as pd

# Establish repo root (notebooks live one level down)
REPO_ROOT = Path.cwd().parent

DATA_DIR = REPO_ROOT / "data/raw/csv"


### Load and Concatenate Files

After extracting the required files from BTS, we'll load and combine them into one dataframe.

In [ ]:
files = sorted(DATA_DIR.glob("*.csv"))
print(f"Found {len(files)} CSV files.") # used for visibility when debugging cwd issues

# Specify columns to load to save memory, reducing columns loaded from 110 to 30 and dataframe memory usage from ~24.8 GB to ~11.1 GB
usecols = [
    # Date / identifiers
    "Year",
    "Month",
    "DayofMonth",
    "DayOfWeek",
    "FlightDate",
    "Reporting_Airline",
    "IATA_CODE_Reporting_Airline",
    "Flight_Number_Reporting_Airline",
    "Tail_Number",

    # Origin / destination
    "Origin",
    "OriginCityName",
    "OriginState",
    "Dest",
    "DestCityName",
    "DestState",

    # Scheduled vs actual times
    "CRSDepTime",
    "CRSArrTime",
    "DepTime",
    "ArrTime",

    # Delay metrics
    "DepDelay",
    "DepDelayMinutes",
    "DepDel15",
    "ArrDelay",
    "ArrDelayMinutes",
    "ArrDel15",

    # Status flags
    "Cancelled",
    "Diverted",
    "CancellationCode",

    # Distance / throughput
    "Distance",
    "Flights",
]

# Adding a dtype map to reduce memory usage further
dtypes = {
    "Year": "int16",
    "Month": "int8",
    "DayofMonth": "int8",
    "DayOfWeek": "int8",
    "Reporting_Airline": "category",
    "IATA_CODE_Reporting_Airline": "category",
    "Tail_Number": "category",
    "Flight_Number_Reporting_Airline": "int32",
    "Origin": "category",
    "OriginCityName": "category",
    "OriginState": "category",
    "Dest": "category",
    "DestCityName": "category",
    "DestState": "category",
    "CRSDepTime": "int32",
    "CRSArrTime": "int32",
    "DepTime": "float32",
    "ArrTime": "float32",
    "DepDelay": "float32",
    "DepDelayMinutes": "float32",
    "DepDel15": "Int8",
    "ArrDelay": "float32",
    "ArrDelayMinutes": "float32",
    "ArrDel15": "Int8",
    "Cancelled": "Int8",
    "Diverted": "Int8",
    "CancellationCode": "category",
    "Distance": "float32",
    "Flights": "Int8",
}

dfs = [pd.read_csv(f, low_memory=False, usecols=usecols, dtype=dtypes) for f in files]

flights = pd.concat(dfs, ignore_index=True)


In [ ]:
len(flights), flights.memory_usage(deep=True).sum() / (1024**3)


#### Find TranStats terminology at:
[BTS Table Info](https://www.transtats.bts.gov/TableInfo.asp?gnoyr_VQ=FGJ&QO_fu146_anzr=b0-gvzr&V0s1_b0yB=D)

### Data Cleaning

BTS column names are in all caps, with mixed formatting, so we'll strip out white space and special characters and replace spaces and hyphens with underscores for standard Python-friendly formatting.

Additionally, these BTS tables report on cancelled and diverted flights. This data will distort KPIs like average delay and on-time rates because they do not have meaningful arrival delay values. Therefore, we will filter them out.


In [ ]:
# Standardize column names

flights.columns = (
    flights.columns.str.strip()  # Remove leading/trailing whitespace
        .str.lower()  # Lowercase all characters
        .str.replace(" ", "_")  # Replace spaces with underscores
)

In [ ]:
# Fill missing flags as 0/1, removing any NaNs that may interfere with filtering

flag_cols = ["cancelled", "diverted", "depdel15", "arrdel15", "flights"]
for c in flag_cols:
    flights[c] = flights[c].fillna(0).astype("int8")


In [ ]:
# Filter Cancelled/Diverted Flights

flights_clean = flights[
    (flights['cancelled'] == 0) &
    (flights['diverted'] == 0)
].copy()

print(f"Rows before cleaning: {len(flights)}")
print(f"Rows after cleaning: {len(flights_clean)}")

<mark>personal note</mark>

.copy() uses a lot of RAM. Flights currently uses ~8G memory, which pushes to 16G when executing above code. If more issues arise due to memory overhead (other apps open and using RAM), then use this code to filter with a mask, selected only needed columns, before you copy:

mask = (flights["cancelled"] == 0) & (flights["diverted"] == 0)
flights_clean = flights.loc[mask].copy()

print(f"Rows before cleaning: {len(flights):,}")
print(f"Rows after cleaning:  {len(flights_clean):,}")


In [ ]:
# Delete flights dataframe to free up memory (and prevent crashes during modeling)
del flights
import gc
gc.collect()

In [ ]:
flights_clean


### Time-based features for modeling

In [ ]:
# Scheduled Departure Hour
flights_clean['dep_hour'] = (flights_clean['crsdeptime'].floordiv(100).astype('int8'))

# Weekend Flag
flights_clean['is_weekend'] = flights_clean['dayofweek'].isin([6, 7]).astype('int8')

# Route feature (control size of feature by using categories to save memory)
flights_clean['route'] = (flights_clean['origin'].astype(str) + "-" + flights_clean['dest'].astype(str)).astype('category')

In [ ]:
# Sanity check feature creation
flights_clean[['arrdel15', 'dep_hour', 'is_weekend', 'route', 'distance']].head()

In [ ]:
# memory check
flights_clean.memory_usage(deep=True).sum() / (1024**3)

### Exploratory Data Analysis (EDA)

#### Dataset sanity check first:

In [ ]:
flights_clean.shape

In [ ]:
# check for distribution, range of hours, distance scale
flights_clean[['arrdel15', 'dep_hour', 'is_weekend', 'route', 'distance']].describe()

#### Class balance: how many flights are late?

In [ ]:
late_rate = flights_clean['arrdel15'].mean()
late_rate

In [ ]:
flights_clean["arrdel15"].value_counts(normalize=True)

19.1% of flights were late, or nearly 1 in 5--normal for airline data. Use as baseline difficulty

#### Delay rate by departure hour (strongest predictor)

In [ ]:
hourly_delay = (
    flights_clean
        .groupby('dep_hour', observed=True)['arrdel15']
        .mean()
        .reset_index()
        .sort_values(by='dep_hour')
)

hourly_delay.head()

In [ ]:
# plot hourly delay rate
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(hourly_delay['dep_hour'], hourly_delay['arrdel15'], marker='o')
plt.title('Arrival Delay Rate by Scheduled Departure Hour')
plt.xlabel('Scheduled Departure Hour')
plt.ylabel('Late Arrival Rate')
plt.show()

**Observations**
1. morning flights have lowest delay rate
2. delay risks climbs throughout the day

#### Weekend v Weekday effect

In [ ]:
weekend_delay = (
    flights_clean
        .groupby('is_weekend', observed=True)['arrdel15']
        .mean()
        .reset_index()
)

weekend_delay

In [ ]:
# label 0/1 for clarity
weekend_delay['day_type'] = weekend_delay['is_weekend'].map({0: 'Weekday', 1: 'Weekend'})
weekend_delay[['day_type', 'arrdel15']]

**Observations**
1. lower delay rates on the weekends
2. feature useful, but not as strong as time of day

#### Carrier-level delay performance (top 10 by volume)

In [ ]:
# top 10 carriers by flight count
top_carriers = (
    flights_clean["iata_code_reporting_airline"]
        .value_counts()
        .head(10)
        .index
)

# calculate average arrival delay rate for top carriers
carrier_delay = (
    flights_clean[flights_clean["iata_code_reporting_airline"].isin(top_carriers)]
    .groupby("iata_code_reporting_airline", observed=True)["arrdel15"]
    .mean()
    .sort_values()
    .reset_index()
)

carrier_delay

#### Route-level delay (top 10 busiest routes)

In [ ]:
# top 10 routes by flight count
top_routes = (
    flights_clean["route"]
        .value_counts()
        .head(10)
        .index
)

top_routes

In [ ]:
# calculate average arrival delay rate for top routes
route_delay = (
    flights_clean[flights_clean["route"].isin(top_routes)]
    .groupby("route", observed=True)["arrdel15"]
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

route_delay

**Observations:**
1. routes that systematically underperform: Chicago→La Guardia ↺, Los Angeles→San Francisco ↺
2. are route-level features useful in model?

#### Distance v delay
sanity check--long flights have buffer to absorb delays, so short flights should show high volatility

In [ ]:
distance_bins = pd.cut(flights_clean['distance'], bins=[0, 500, 1000, 1500, 2000, 2500, 3000, 4000, 5000])

distance_delay = (
    flights_clean
        .groupby(distance_bins, observed=True)['arrdel15']
        .mean()
        .reset_index()
)

distance_delay

In [ ]:
distance_delay = (
    flights_clean
        .groupby(distance_bins, observed=True)['arrdelay']
        .mean()
        .reset_index()
)

distance_delay

**Observation**
I can see the following breakdown in delay rates:
- short-haul (0 - 500): ~18.2%
- mid-haul (500-1500): ~19-20%
- long-haul (1500-3000): ~20%
- very long-haul (3000+): ~16.2%
long-haul flights have built-in schedule buffers, priority handling, reduced connection dependencies that reduce delayed arrival probability. The lower rate is expected. Short flights must handle faster turn around, congested airspace, and gate delays that have greater affect on crossing the 15 minute threshold.

In [ ]:
# volatility for short vs long hauls actually shows up in the stdev of delay rates
distance_delay_stdev = (
    flights_clean
        .groupby(distance_bins, observed=True)['arrdelay']
        .std()
        .reset_index()
)

distance_delay_stdev

In [ ]:
distance_delay_iqr = (
    flights_clean
        .groupby(distance_bins, observed=True)['arrdelay']
        .quantile([0.25, 0.75])
        .unstack()
)

distance_delay_iqr["iqr"] = distance_delay_iqr[0.75] - distance_delay_iqr[0.25]
distance_delay_iqr.reset_index()

#### Memory cleanup
...or, next time upgrade to the bigger memory package! 🧘🏻‍♀️

In [ ]:
del hourly_delay, weekend_delay, carrier_delay, route_delay, distance_delay, distance_delay_iqr, distance_delay_stdev
import gc
gc.collect()


### Modeling
#### Logistics Regression

**Target:** `arrdel15` (0 = on time, 1 = ≥15 min late)

**Baseline Features:** `dep_hour`, `is_weekend`, `distance`, `iata_code_reporting_airline` (EDA above proved that these matter)

In [ ]:
# Prepare data for modeling
model_features = [
    'dep_hour',
    'is_weekend',
    'distance',
    'iata_code_reporting_airline'
]

X = flights_clean[model_features].copy()
Y = flights_clean['arrdel15']


In [ ]:
# Split data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    Y,
    test_size=0.2,
    random_state=42,
    stratify=Y # keeps late/on-time ratio consistent across train/test splits
)

In [ ]:
# Preprocess pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

numeric_features = ['dep_hour', 'distance'] # scaled
binary_features = ['is_weekend'] # no scaling needed
categorical_features = ['iata_code_reporting_airline'] # one-hot encoded due to categorical nature

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('bin', 'passthrough', binary_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=True), categorical_features)
    ]
)

In [ ]:
# Logistic Regression Model
model = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(
            max_iter=1000, 
            class_weight='balanced'
        )),
    ]
)

In [ ]:
model.fit(X_train, Y_train)

In [ ]:
# Model Performance Evaluation
from sklearn.metrics import classification_report, roc_auc_score

Y_pred = model.predict(X_test)
Y_proba = model.predict_proba(X_test)[:, 1]

print(classification_report(Y_test, Y_pred))
print(f"ROC AUC Score: {roc_auc_score(Y_test, Y_proba)}")

In [ ]:
import numpy as np

feature_names = (
    model.named_steps['preprocessor']
        .get_feature_names_out()
)

coefs = model.named_steps['classifier'].coef_[0]

coef_df = (
    pd.DataFrame({
        'feature': feature_names,
        'coefficient': coefs
    })
    .sort_values('coefficient', ascending=False) 
)

coef_df.head(10)

In [ ]:
coef_df.tail(10)

#### Gradient Boosting Decision Tree-model

**Target:** `arrdel15` (0 = on time, 1 = ≥15 min late)

**Baseline Features:** `dep_hour`, `is_weekend`, `distance`, `iata_code_reporting_airline` (EDA above proved that these matter)

*Why Gradient Boosting?* Gradient Boosting has better performance on structure data and uses less memory (compared with Random Forest). `HistGradientBoostingClassifier` is fast, scales well, handles large datasets, and does not require one-hot encoding.

In [ ]:
# Create feature set for tree-modeling
X_tree = flights_clean[model_features].copy()
Y_tree = flights_clean['arrdel15']

In [ ]:
# Encode categorical features (Ordinal)
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_tree['iata_code_reporting_airline'] = encoder.fit_transform(
    X_tree[['iata_code_reporting_airline']]
)

In [ ]:
# Train/test split for tree model
X_tree_train, X_tree_test, Y_tree_train, Y_tree_test = train_test_split(
    X_tree,
    Y_tree,
    test_size=0.2,
    random_state=42,
    stratify=Y_tree
)

In [ ]:
# Train Gradient Boosting Classifier
from sklearn.ensemble import HistGradientBoostingClassifier

gb_model = HistGradientBoostingClassifier(
    max_depth=6, # avoids overfitting
    learning_rate=0.05, # slower learning for better generalization
    max_iter=150,
    class_weight='balanced', # handle class imbalance and consistent with baseline (previous lr model)
    random_state=42
)

gb_model.fit(X_tree_train, Y_tree_train)

In [ ]:
# Evaluate Tree Model

Y_tree_pred = gb_model.predict(X_tree_test)
Y_tree_proba = gb_model.predict_proba(X_tree_test)[:, 1]

print(classification_report(Y_tree_test, Y_tree_pred))
print(f"Tree Model ROC AUC Score: {roc_auc_score(Y_tree_test, Y_tree_proba)}")

In [ ]:
# Compare LR vs Tree Model

comparison_df = pd.DataFrame({
    'Model': ['Logistic Regression', 'Gradient Boosting'],
    'ROC AUC Score': [
        roc_auc_score(Y_test, Y_proba), 
        roc_auc_score(Y_tree_test, Y_tree_proba)
    ],
})

comparison_df

In [ ]:
# Feature Importance for Tree Model

from sklearn.inspection import permutation_importance

# use sample of test set for efficiency
# n=min(200000, len(X_tree_test))
sample_idx = X_tree_test.sample(n=200_000, random_state=42).index

perm = permutation_importance(
    gb_model, 
    X_tree_test.loc[sample_idx], 
    Y_tree_test.loc[sample_idx],
    n_repeats=5,
    random_state=42,
    n_jobs=-1
)

perm_importance_df = ({
    'feature': X_tree.columns,
    'importance': perm.importances_mean
}).sort_values(by='importance', ascending=False)

perm_importance_df

#### Model Comparison
- logistics regression model was used as an interpretable baseline
- gradient boosting classifier was trained to capture nonlinear and interaction effects
- both models were trained on identical features (`model_features`) and evaluated on the same test set.

#### Results Summary
- logistic regression provides strong interpretability and stable performance
- gradient boosting improves ROC AUC by 1.7 points, indicating a better ranking of delay risk (thousands of flights from the 14.6 million in our dataset)
- the strongest predictor in both models is scheduled departure hour, confirming operational delay propogation.

#### **Recommendation**
- use **logistic regression** for explainability and policy decisions
- use **gradient boosting** for operational risk scoring where performance is a priority
- both models agree on key drivers, which increases confidence in insights